In [27]:
# coding:utf-8
import requests
import csv
import pandas as pd
from bs4 import BeautifulSoup
from requests.exceptions import RequestException

In [28]:
def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) ' + 
            'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'
        }
        html = requests.get(url, headers=headers)
        if html.status_code == 200:
            return html.text
        return None
    except RequestException:
        return None

In [29]:
result_all = [] # 用于存储样本
 
def parse_one_page(html):
    soup = BeautifulSoup(html, 'lxml')
    companies = soup.find_all('div', 'job-primary', True)
    
    for com in companies:
        res = parse_one_company(com)
        result_all.append(res)

In [30]:
def parse_one_company(comp):
    result = []
    company_soup = comp.find('div', class_='info-company')
    com_desc = company_soup.find('p').text
 
    primary_soup = comp.find('div', class_='info-primary')
    job_name = primary_soup.find('div').text
    salary = primary_soup.find('span').text
    requirement = primary_soup.find('p').text
 
    result.append(com_desc)
    result.append(job_name)
    result.append(salary)
    result.append(requirement)
    return result

In [31]:
def parse_all_page(num, offset):
    url1 = 'https://www.zhipin.com/c100010000/?query=数据分析师&page='+str(offset)+'&ka=page-'+str(offset)
    #url1 = 'https://www.zhipin.com/c101280100/h_101280100/?query=数据分析师&page='+str(offset)+'&ka=page-'+str(offset) # 广州
    #url2 = 'https://www.zhipin.com/c101280600/h_101280600/?query=数据分析师&page='+str(offset)+'&ka=page-'+str(offset) # 深圳
    #url3 = 'https://www.zhipin.com/c101010100/h_101010100/?query=数据分析师&page='+str(offset)+'&ka=page-'+str(offset) # 北京
    #url4 = 'https://www.zhipin.com/c101020100/h_101020100/?query=数据分析师&page='+str(offset)+'&ka=page-'+str(offset) # 上海
    urldict = {'1':url1}
 
    html = get_one_page(urldict[str(num)])
    parse_one_page(html)

 
 
if __name__ == '__main__':
    for j in range(1, 2):
        for i in range(1,20):
            parse_all_page(j, i)
    file = pd.DataFrame(result_all, columns=['公司信息', '岗位', '薪水', '其他'])
    # encoding='utf_8_sig解决保存到CSV文件后显示乱码问题
    file.to_csv('Bosszhiping1.csv', mode='a', index=True, encoding='utf_8_sig')

In [26]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import re
 
# 导入数据，第一列的序号不导入
data_df = pd.read_csv('Bosszhiping1.csv', usecols=[1,2,3,4])
 
# 查看数据前5行信息，简单了解下数据
data_df.head(10)

,公司信息,岗位,薪水,其他
0,游戏已上市1000-9999人,数据分析,15k-25k,北京 朝阳区 小营经验不限本科
1,游戏已上市10000人以上,数据分析师,13k-25k,广州 天河区 天河公园1-3年本科
2,O2OD轮及以上10000人以上,数据分析,13k-26k,上海 普陀区 长征3-5年本科
3,互联网C轮1000-9999人,数据分析师,10k-20k,杭州 余杭区 仓前3-5年硕士
4,移动互联网D轮及以上10000人以上,数据分析,6k-11k,杭州 1-3年本科
5,移动互联网D轮及以上1000-9999人,数据分析,4k-6k,南京 雨花台区 铁心桥经验不限大专
6,计算机软件已上市10000人以上,数据分析师,9k-11k,上海 1-3年本科
7,在线教育不需要融资100-499人,数据分析,4k-8k,北京 大兴区 黄村经验不限本科
8,移动互联网A轮1000-9999人,数据分析,5k-8k,武汉 洪山区 光谷1-3年大专
9,移动互联网已上市1000-9999人,数据分析,4k-6k,成都 经验不限本科


In [35]:
others = list(data_df['其他'])
# 对edu列表进一步解析，得到“工作经验”和“学历”特征
leng = len(others)
for i in range(leng):
    others[i] = others[i].replace('应届生','0-0 ')
    others[i] = others[i].replace('经验不限', '0-n ')
    others[i] = others[i].replace('年', ' ') 
    others[i] = others[i].replace('区 ', '区')
    others[i] = others[i].replace('10', '9')    

# 工作经验和学历没有空格隔开，我们暂时将两者放到edu列表中
city, exp, edu= [], [], []
leng = len(others)
for s in others:
    temp = s.split(' ')
    city.append(temp[0])
    edus=s[-2:]
    edu.append(edus)
    exps=s[-6:-2]
    exp.append(exps)
data_df['地区']=city
data_df['学历']=edu
data_df['经验']=exp

# 对edu列表进一步解析，得到“工作经验”和“学历”特征
exp=data_df['经验']
leng = len(exp)
for i in range(leng):
    exp[i] = exp[i].replace('9', '10')
    exp[i] = exp[i].replace('以内', '')  
salary = list(data_df['薪水'])
salary_low,salary_up,salary_all=[],[],[]

for s in salary:
    t = s.split('k')
    salary_all.append(t)
data_df['薪水1']=salary_all    
data_df.head()

,公司信息,岗位,薪水,其他,地区,学历,经验,薪水1
0,游戏已上市1000-9999人,数据分析,15k-25k,北京 朝阳区 小营经验不限本科,北京,本科,0-n,"[15, -25, ]"
1,游戏已上市10000人以上,数据分析师,13k-25k,广州 天河区 天河公园1-3年本科,广州,本科,1-3,"[13, -25, ]"
2,O2OD轮及以上10000人以上,数据分析,13k-26k,上海 普陀区 长征3-5年本科,上海,本科,3-5,"[13, -26, ]"
3,互联网C轮1000-9999人,数据分析师,10k-20k,杭州 余杭区 仓前3-5年硕士,杭州,硕士,3-5,"[10, -20, ]"
4,移动互联网D轮及以上10000人以上,数据分析,6k-11k,杭州 1-3年本科,杭州,本科,1-3,"[6, -11, ]"


In [36]:
# 生成文件
data_df.to_excel('boss.xls', index=True, encoding='utf_8_sig')